# core

> Generate Markdown Files From A Python Library

In [ ]:
#| default_exp core

In [ ]:
#|export
import pkgutil, pydoc, re
from importlib import import_module
from types import ModuleType
from pathlib import Path
from fastcore.script import call_parse
from fastcore.xtras import mk_write

In [ ]:
#|export
def get_modules(lib:ModuleType) -> list[str]:
    "get a list of modules from a python package"
    modules = []
    for _, modname, _ in pkgutil.iter_modules(lib.__path__, lib.__name__ + '.'):
        if not modname.split('.')[-1].startswith('_'): modules.append(modname)
    return modules

For example, we can list all the modules in the [requests](https://requests.readthedocs.io/en/latest/) library:

In [ ]:
import requests

In [ ]:
mods = get_modules(requests)
mods

['requests.adapters',
 'requests.api',
 'requests.auth',
 'requests.certs',
 'requests.compat',
 'requests.cookies',
 'requests.exceptions',
 'requests.help',
 'requests.hooks',
 'requests.models',
 'requests.packages',
 'requests.sessions',
 'requests.status_codes',
 'requests.structures',
 'requests.utils']

In [ ]:
#|hide
assert mods == ['requests.adapters','requests.api', 'requests.auth', 'requests.certs', 'requests.compat', 'requests.cookies',
 'requests.exceptions', 'requests.help', 'requests.hooks', 'requests.models', 'requests.packages', 'requests.sessions',
 'requests.status_codes', 'requests.structures', 'requests.utils']

In [ ]:
#|export
def gethelp(modname:str)->str:
    "Get the help string for a module, when the module is a string."
    sym = __import__(modname, fromlist=[''])
    return pydoc.render_doc(sym, title='~~~~~~~~~~~~%s', renderer=pydoc.plaintext)

In [ ]:
txt = gethelp('requests.api')
txt[:200]

'~~~~~~~~~~~~module requests.api in requests\n\nNAME\n    requests.api\n\nDESCRIPTION\n    requests.api\n    ~~~~~~~~~~~~\n    \n    This module implements the Requests API.\n    \n    :copyright: (c) 2012 by Ken'

In [ ]:
#|export
def help2md(helpstr):
    "Transform a help string for a module into Markdown."
    md = []
    section = None
    class_prefix = re.compile('^     \|  ')

    for l in helpstr.splitlines():
        if l.strip().startswith('~~~~~~~~~~~~'): continue

        if l and re.search('^\w', l): 
            section = l.strip().lower()
            if section not in ['name', 'data', 'file', 'version']: md.append(f'## {l.title()}')
            continue
        
        if section == 'description' and '[[quarto_pydoc:ignore]]' in l: return ''
        if section in ['file', 'data', 'version']: continue

        if l and re.search('^    \w', l):
            if section == 'name': md.append(f'# {l.strip()}')
            if section == 'functions': md.extend(['', f"### {l.split('(')[0].strip()}", '', f'<strong>{l.strip()}</strong>'])
            if section == 'classes': 
                if l.startswith('    class '): 
                    md.extend(['', f"### {l.replace('    class ', '').split('(')[0].strip()}", '', f'<strong>{l.strip()}</strong>', ''])
                else: md.append(l)
            continue

        else: md.append(l)
    return '\n'.join(md)

This does some minimal transformation on the text to create markdown headings:

In [ ]:
print(help2md(txt))


# requests.api

## Description
    
    
    :copyright: (c) 2012 by Kenneth Reitz.
    :license: Apache2, see LICENSE for more details.

## Functions

### delete

<strong>delete(url, **kwargs)</strong>
        Sends a DELETE request.
        
        :param url: URL for the new :class:`Request` object.
        :param \*\*kwargs: Optional arguments that ``request`` takes.
        :return: :class:`Response <Response>` object
        :rtype: requests.Response
    

### get

<strong>get(url, params=None, **kwargs)</strong>
        Sends a GET request.
        
        :param url: URL for the new :class:`Request` object.
        :param params: (optional) Dictionary, list of tuples or bytes to send
            in the query string for the :class:`Request`.
        :param \*\*kwargs: Optional arguments that ``request`` takes.
        :return: :class:`Response <Response>` object
        :rtype: requests.Response
    

### head

<strong>head(url, **kwargs)</strong>
        Sends a HEAD request

In [ ]:
#|export
@call_parse
def gen_md(lib:str, # the name of the python library
           dest_dir:str # the destination directory the markdown files will be rendered into
          ) -> None:
    "Generate Markdown API docs"
    for modname in get_modules(import_module(lib)):  
        helpstr = gethelp(modname)
        md = help2md(helpstr)
        if md == '': continue
        submod = modname.split('.')[-1]
        (Path(dest_dir)/f'{submod}.md').mk_write(md)

You can generate your docs in the desired directory like so:

In [ ]:
!rm -rf _test_dir/
gen_md('requests', '_test_dir/')

In [ ]:
!ls _test_dir

adapters.md     certs.md        exceptions.md   models.md       status_codes.md
api.md          compat.md       help.md         packages.md     structures.md
auth.md         cookies.md      hooks.md        sessions.md     utils.md


In [ ]:
#|hide
assert len(mods) == len(Path('_test_dir/').ls())